In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import numpy as np
from time import time
from sklearn.datasets import load_digits

In [2]:
from flaml.data import load_openml_dataset
X_train, X_test, y_train, y_test = load_openml_dataset(dataset_id=1169, data_dir='./')

load dataset from ./openml_ds1169.pkl
Dataset name: airlines
X_train.shape: (404537, 7), y_train.shape: (404537,);
X_test.shape: (134846, 7), y_test.shape: (134846,)


In [3]:
data = pd.read_csv("datasets/income.csv")
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income >50K
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


In [4]:
data["education"] = data["education"].replace(
    {
        'Preschool': 'dropout',
        '10th': 'dropout',
        '11th': 'dropout',
        '12th': 'dropout',
        '1st-4th': 'dropout',
        '5th-6th': 'dropout',
        '7th-8th': 'dropout',
        '9th': 'dropout',
        'HS-Grad': 'HighGrad',
        'HS-grad': 'HighGrad',
        'Some-college': 'CommunityCollege',
        'Assoc-acdm': 'CommunityCollege',
        'Assoc-voc': 'CommunityCollege',
        'Bachelors': 'Bachelors',
        'Masters': 'Masters',
        'Prof-school': 'Masters',
        'Doctorate': 'Doctorate'
    }
)
data["marital-status"] = data["marital-status"].replace(
    {
        'Never-married': 'NotMarried',
        'Married-AF-spouse': 'Married',
        'Married-civ-spouse': 'Married',
        'NotMarried': 'NotMarried',
        'Separated': 'Separated',
        'Divorced': 'Separated',
        'Widowed': 'Widowed',
    }
)
ordinal_columns={
    "education": ["dropout", "HighGrad", "CommunityCollege", "Bachelors", "Masters", "Doctorate"],
}
data["native-country"] = data["native-country"].replace(
{"Canada": "North America", "Cuba": "South America", "Dominican-Republic": "South America", "El-Salvador": "South America", "Guatemala": "South America",
                   "Haiti": "South America", "Honduras": "South America", "Jamaica": "South America", "Mexico": "South America", "Nicaragua": "South America",
                   "Outlying-US(Guam-USVI-etc)": "North America", "Puerto-Rico": "North America", "Trinadad&Tobago": "South America",
                   "United-States": "North America", "Cambodia": "Asia", "China": "Asia", "Hong": "Asia", "India": "Asia", "Iran": "Asia", "Japan": "Asia", "Laos": "Asia",
          "Philippines": "Asia", "Taiwan": "Asia", "Thailand": "Asia", "Vietnam": "Asia", "Columbia": "South America", "Ecuador": "South America", "Peru": "South America", "England": "West Europe", "France": "West Europe", "Germany": "West Europe", "Greece": "West Europe", "Holand-Netherlands": "West Europe",
            "Ireland": "West Europe", "Italy": "West Europe", "Portugal": "West Europe", "Scotland": "West Europe", "Hungary": "East Europe", "Poland": "East Europe", "Yugoslavia": "East Europe", "South": "?"})
data["capital"] = data["capital-gain"]-data["capital-loss"]
data.drop(["capital-gain", "capital-loss"], axis=1, inplace=True)
data["education-num"] = data["education-num"].astype(float)

In [5]:
from sklearn.model_selection import RepeatedStratifiedKFold
from automl.search.automl import AutoML

am = AutoML(
    random_state=42,
    level="common",
    #target_metric="f1",
    trainer_config={
        #"secondary_level": ComponentLevel.UNCOMMON,
        "cache": True,
        "early_stopping": True,
        "return_test_scores_during_tuning": True,
        "tuning_time": 60*5,
        "stacking_level": 0,
        "tune_kwargs": {"max_concurrent": 2, "trainable_n_jobs": 4}
    },
    #cv=RepeatedStratifiedKFold(random_state=420, n_repeats=2, n_splits=4)
    cv=4,
)

/home/ubuntu/.local/share/virtualenvs/automl-R4LDggHl/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [6]:
t_s = time()
am.fit(data.drop("income >50K", axis=1), data["income >50K"], ordinal_columns={"education":ordinal_columns})
#am.fit(data.drop("Purchase", axis=1), data["Purchase"])
#am.fit(data.drop("default", axis=1), data["default"], ordinal_columns={"EDUCATION": list(sorted(data["EDUCATION"].unique()))})
#am.fit(data.drop("quality", axis=1), data["quality"]>=6)
#am.fit(data.drop("target", axis=1), data["target"])
#am.fit(data.drop("target", axis=1), data["target"], ordinal_columns=ordinal_columns)
#am.fit(X_train, y_train, X_test=X_test, y_test=y_test)
t_end = time() - t_s

``multivariate`` option is an experimental feature. The interface can change in the future.
``group`` option is an experimental feature. The interface can change in the future.
[I 2021-07-08 19:02:56,260] A new study created in memory with name: optuna


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best validation score',
         …

Passing `extra_*` resource requirements to `resources_per_trial` is deprecated. Please use a `PlacementGroupFactory` object to define your resource requirements instead.


suggest 1cc7721c, 12
1cc7721c final suggestion by 0: {'Estimator': 'RandomForestClassifier(n_estimators=100, randomization_type=rf, criterion=gini, max_depth=10, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=1e-10, max_features=0.2, random_state=0, max_leaf_nodes=None, min_impurity_decrease=1e-10, bootstrap=True, oob_score=False, n_jobs=None, class_weight=balanced, ccp_alpha=0.0, verbose=0, warm_start=False, max_samples=None)', 'FeatureSelector': 'Passthrough(None)', 'Estimator__RandomForestClassifier__n_estimators': 100, 'Estimator__RandomForestClassifier__randomization_type': 'et', 'Estimator__RandomForestClassifier__criterion': 'gini', 'Estimator__RandomForestClassifier__min_samples_split': 2, 'Estimator__RandomForestClassifier__min_samples_leaf': 1, 'Estimator__RandomForestClassifier__max_features': 0.2, 'Estimator__RandomForestClassifier__max_depth': 10, 'Estimator__RandomForestClassifier__min_impurity_decrease': 1e-10, 'Estimator__RandomForestClassifier__min_w

suggest 1cc7721d, 11
1cc7721d final suggestion by 0: {'Estimator': 'CatBoostClassifierBinary(n_estimators=200, learning_rate=None, max_depth=6, task_type=CPU, verbose=False, random_state=None, auto_class_weights=Balanced)', 'FeatureSelector': 'Passthrough(None)', 'Estimator__CatBoostClassifierBinary__n_estimators': 200, 'Estimator__CatBoostClassifierBinary__max_depth': 6, 'Estimator__CatBoostClassifierBinary__auto_class_weights': 'Balanced', 'Imputer__CombinedSimpleImputer__categorical_strategy': 'most_frequent', 'ColumnScaling__ScalerNormalizer': 'passthrough', 'Imputer': 'CombinedSimpleImputer(numeric_strategy=median, numeric_fill_value=0, categorical_strategy=most_frequent, categorical_fill_value=missing_value, verbose=0, copy=True, n_jobs=1, transformer_weights=None)', 'Imbalance': 'Passthrough(None)', 'ColumnOrdinal__OrdinalEncoder': "OrdinalEncoder(categories=auto, handle_unknown=use_encoded_value, dtype=<class 'numpy.uint16'>, unknown_value=None)", 'ColumnEncoding__BinaryEncoder

...

scoring took 9.955959558486938
scoring ensemble
estimator PandasStackingClassifier n_jobs: 1
using saved predictions
using saved predictions


/home/ubuntu/automl/automl_models/components/estimators/ensemble/stack.py:186: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/ubuntu/automl/automl_models/components/estimators/ensemble/stack.py:186: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/ubuntu/automl/automl_models/components/estimators/ensemble/stack.py:186: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/ubuntu/automl/automl_models/components/estimators/ensemble/stack.py:186: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/ubuntu/automl/automl_models/components/estimators/ensemble/stack.py:186: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future wi

scoring took 1.6869511604309082


/home/ubuntu/automl/automl_models/components/estimators/ensemble/stack.py:186: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

/home/ubuntu/automl/automl_models/components/estimators/ensemble/stack.py:193: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,Pipeline,Test accuracy,Test balanced_accuracy,Test roc_auc,Test precision,Test recall,Test specificity,Test f1,Test matthews_corrcoef,Test optimized_precision,...,Validation precision,Validation recall,Validation specificity,Validation f1,Validation matthews_corrcoef,Validation optimized_precision,Stacking Level,Total Time (s),Estimator Fit Time (s),Dataset Fraction
Id,,,,,,,,,,,,,,,,,,,,,
0_VotingSoftByMetric,PandasVotingClassifier,0.850264,0.845819,0.929663,0.645809,0.837245,0.854392,0.729171,0.638175,0.840127,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0
0_VotingSoft,PandasVotingClassifier,0.851615,0.846186,0.929595,0.648970,0.835714,0.856657,0.730598,0.640029,0.839240,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0
ca912b90,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844000,0.839951,0.923707,0.634137,0.832143,0.847759,0.719771,0.625287,0.834703,...,0.634515,0.790511,0.855548,0.703968,0.602800,0.800375,0,9.063558,5.483729,1.0
0_VotingUniform,PandasVotingClassifier,0.844122,0.848568,NaN,0.629449,0.857143,0.839994,0.725859,0.634933,0.834017,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0
98bc3dc6,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.842648,0.847075,0.928648,0.626916,0.855612,0.838537,0.723625,0.631850,0.832570,...,0.623820,0.814658,0.844112,0.706546,0.606677,0.819262,0,4.088792,3.175313,1.0
0_VotingByMetric,PandasVotingClassifier,0.843508,0.848861,NaN,0.627890,0.859184,0.838537,0.725549,0.634678,0.831347,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0
55d71f6c,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.841788,0.847554,0.927832,0.624722,0.858673,0.836434,0.723249,0.631585,0.828669,...,0.621453,0.819759,0.841523,0.706918,0.607356,0.823181,0,9.833414,5.757895,1.0
5210fd94,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.842648,0.834532,0.922033,0.634137,0.818878,0.850186,0.714763,0.618013,0.823890,...,0.632922,0.781668,0.856141,0.699445,0.596612,0.792735,0,6.642378,4.448233,1.0
9607ef1c,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.839209,0.847249,0.929377,0.619187,0.862755,0.831742,0.720955,0.628881,0.820907,...,0.617134,0.824860,0.837586,0.705989,0.606307,0.826866,0,8.250609,5.158554,1.0


In [23]:
a = am.get_pipeline_by_id("91a28710")

Finished loading model, total used 66 iterations


In [24]:
a.steps.pop(-1)

('Estimator',
 LogisticRegression(class_weight='balanced', l1_ratio=0.5, max_iter=200,
                    penalty='elasticnet', random_state=42, solver='saga'))

In [25]:
x = a.fit_transform(am.X_, am.y_)

In [26]:
x

,marital-status,occupation,relationship,age,education-num,capital
19359,-0.124056,-0.473291,-0.008957,-0.473684,0.000000,0.0
15003,0.879363,0.151460,0.895744,-0.105263,0.666667,0.0
12490,-0.121258,-0.715864,-0.173605,-0.684211,0.000000,0.0
32292,0.873204,-0.174355,0.886522,0.210526,0.000000,0.0
5163,-0.121809,0.151460,-0.247393,-0.526316,1.333333,0.0
...,...,...,...,...,...,...
31450,0.884189,0.535585,0.898115,0.947368,1.333333,0.0
20560,-0.001838,0.497922,-0.113345,0.105263,0.000000,0.0
23650,0.884189,1.175944,0.898115,0.947368,-0.333333,0.0
25301,0.879363,1.208270,0.895744,-0.578947,1.000000,0.0


In [10]:
x.dtypes

marital-status    float32
occupation        float32
relationship      float32
age               float32
education-num     float32
hours-per-week    float32
capital           float32
dtype: object

In [27]:
am.X_test_

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,capital
17581,49,Self-emp-not-inc,Some-college,10,Divorced,Exec-managerial,Not-in-family,Amer-Indian-Eskimo,Female,40,United-States,0
16280,27,Private,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,40,United-States,0
24719,33,Private,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,41,United-States,0
3986,30,Private,HS-grad,9,Separated,Adm-clerical,Not-in-family,White,Female,40,United-States,0
5138,41,Federal-gov,Some-college,10,Never-married,Protective-serv,Own-child,White,Male,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6837,23,State-gov,12th,8,Never-married,Exec-managerial,Not-in-family,White,Male,40,United-States,0
7133,21,Private,7th-8th,4,Never-married,Farming-fishing,Unmarried,White,Male,40,United-States,0
25363,24,Private,Some-college,10,Never-married,Sales,Other-relative,White,Male,35,United-States,0
13085,44,State-gov,HS-grad,9,Married-civ-spouse,Protective-serv,Husband,White,Male,38,United-States,0


In [13]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

[CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42)]

In [14]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

[CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42),
 CatBoostEncoder(cols=[], random_state=42)]

In [ ]:
a.score(am.X_test_, am.y_test_)

In [12]:
am.visualize_pipeline("65a71b94")

Finished loading model, total used 313 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 313 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 200 iterations
Finished loading model, total used 91 iterations


In [11]:
am.results_.sort_values("Test accuracy", ascending=False)[:60]

,Pipeline,Test accuracy,Test balanced_accuracy,Test roc_auc,Test precision,Test recall,Test specificity,Test f1,Test matthews_corrcoef,Test optimized_precision,...,Validation precision,Validation recall,Validation specificity,Validation f1,Validation matthews_corrcoef,Validation optimized_precision,Stacking Level,Total Time (s),Estimator Fit Time (s),Dataset Fraction
Id,,,,,,,,,,,,,,,,,,,,,
4de38d26,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.874831,0.809212,0.929950,0.771182,0.682653,0.935771,0.724222,0.645655,0.718433,...,0.772099,0.653461,0.938778,0.707789,0.628628,0.690873,1,7.030053,5.999565,1.0
3803c584,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.873603,0.805442,0.919485,0.772063,0.673980,0.936903,0.719695,0.641040,0.710386,...,0.574575,0.880123,0.793193,0.695221,0.596715,0.762177,1,7.377254,5.904064,1.0
65a71b94,"LogisticRegression w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasSHAPSelectFromModel, PandasOrdinalEncoder, KFoldCatBoostEncoder, PandasStandardScaler, PandasQuantileTransformer",0.871760,0.826701,0.932272,0.730847,0.739796,0.913606,0.735294,0.650703,0.766637,...,0.732808,0.720456,0.916662,0.726576,0.640848,0.749562,1,35.281595,19.560393,1.0
e50551d0,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.870655,0.799842,0.930195,0.767868,0.663265,0.936418,0.711744,0.631740,0.699900,...,0.768591,0.641558,0.938724,0.699332,0.619113,0.679112,1,60.473898,49.453166,1.0
118d399a,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.859722,0.768079,0.915299,0.772667,0.591327,0.944831,0.669942,0.591272,0.629600,...,0.768893,0.593946,0.943363,0.670176,0.590405,0.631923,0,6.619399,5.768160,1.0
57b742d4,"RandomForestExtraTreesClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder, KFoldCatBoostEncoder",0.854809,0.842366,0.905205,0.660082,0.818367,0.866365,0.730752,0.639763,0.826319,...,0.665415,0.791526,0.872537,0.721929,0.628525,0.804347,1,21.710557,12.963434,1.0
77528d16,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.845105,0.823084,0.917290,0.648030,0.780612,0.865556,0.708169,0.608823,0.793504,...,0.642489,0.737630,0.869788,0.686774,0.580699,0.755743,0,17.626041,11.079301,1.0
1_VotingSoft,PandasVotingClassifier,0.845105,0.850783,0.932678,0.630459,0.861735,0.839832,0.728174,0.638358,0.832233,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0
9a39f0f8,"LGBMClassifier w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.844982,0.847915,0.927916,0.631798,0.853571,0.842259,0.726128,0.635034,0.838311,...,0.625104,0.813977,0.845083,0.707108,0.607423,0.818843,0,17.043041,10.464772,1.0


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

lr = LogisticRegression(random_state=42).fit(X_train, y_train)
dt = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

/home/baum/.local/share/virtualenvs/automl-KTV6y-vU/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [2]:
predictions = [lr.predict_proba(X_train), dt.predict_proba(X_train)]

In [3]:
from automl_models.components.estimators.ensemble import PandasStackingClassifier
stack = PandasStackingClassifier(estimators=[("lr", lr), ("dt", dt)], final_estimator=LogisticRegression(random_state=42)).fit(X_train, y_train)

fitting estimator LogisticRegression(random_state=42)
fitting estimator DecisionTreeClassifier(random_state=42)
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
doing cv for LogisticRegression(random_state=42).predict_proba
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED 

In [4]:
p = stack.predict_proba(X_test)

In [5]:
stack._saved_test_predictions=predictions
pp = stack.predict_proba(X_test)

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [6]:
p == pp

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [6]:
am.results_.loc["0_VotingUniform"]

Pipeline                          PandasVotingClassifier
Test accuracy                                   0.832944
Test balanced_accuracy                          0.848349
Test roc_auc                                         NaN
Test precision                                  0.605559
Test recall                                     0.878061
Test specificity                                0.818638
Test f1                                         0.716785
Test matthews_corrcoef                          0.624876
Test optimized_precision                        0.797921
Validation accuracy                                  NaN
Validation balanced_accuracy                         NaN
Validation roc_auc                                   NaN
Validation precision                                 NaN
Validation recall                                    NaN
Validation specificity                               NaN
Validation f1                                        NaN
Validation matthews_corrcoef   

In [7]:
p = am.get_pipeline_by_id("0_VotingUniform")

Finished loading model, total used 200 iterations
Finished loading model, total used 200 iterations


In [8]:
p.score(am.X_test_, am.y_test_)

0.8329443557302543

In [16]:
p._final_estimator

RandomForestClassifier(class_weight='balanced_subsample', max_depth=7,
                       max_features=0.9041555101461939,
                       min_impurity_decrease=3.384914384121488e-07,
                       min_samples_leaf=20, min_samples_split=19,
                       min_weight_fraction_leaf=1.7645447745993082e-06,
                       n_estimators=221, random_state=42)

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, clone, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from deslib.des import KNORAE, METADES, KNORAU
import numpy as np

In [2]:
rfc = RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True)
X, y = load_breast_cancer(return_X_y=True)


In [234]:
data = pd.read_csv("datasets/wine.csv")

In [246]:
X = data.drop("type", axis=1).to_numpy()
y = LabelEncoder().fit_transform(data["type"].to_numpy())

In [5]:
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

In [15]:
class DESCV(ClassifierMixin, BaseEstimator):
    def __init__(self, des, classifier_pool, cv):
        self.des = des
        self.classifier_pool = classifier_pool
        self.cv = cv
    def fit(self, X, y):
        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        transformed_y = self.le_.transform(y)

        des = []
        for train_ind, test_ind in self.cv.split(X, y):
            rfc_ = clone(self.classifier_pool).fit(X[train_ind], y[train_ind])
            des_ = clone(self.des)
            des_.set_params(pool_classifiers=rfc_)
            des_.fit(X[test_ind], y[test_ind])
            des.append(des_)
        self.estimators_ = des

    def _predict(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([est.predict(X) for est in self.estimators_]).T

    def _collect_probas(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([clf.predict_proba(X) for clf in self.estimators_])

    def predict(self, X):
        maj = np.argmax(self.predict_proba(X), axis=1)
        maj = self.le_.inverse_transform(maj)

        return maj

    def predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting."""
        #check_is_fitted(self)
        avg = np.average(self._collect_probas(X), axis=0)
        return avg

In [7]:
X, y = make_classification(n_samples=7000, random_state=42)

In [16]:
descv = DESCV(KNORAE(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [17]:
desucv = DESCV(KNORAU(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [18]:
metadescv = DESCV(METADES(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [11]:
%%time
np.mean(cross_val_score(rfc, X, y, scoring="roc_auc_ovr_weighted"))

CPU times: user 10.4 s, sys: 16.8 ms, total: 10.5 s
Wall time: 10.5 s


0.991752412179687

In [261]:
np.mean(cross_val_score(rfc, X, y, scoring="f1", error_score="raise"))

0.9792307757731944

In [19]:
%%time
np.mean(cross_val_score(descv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

CPU times: user 48.5 s, sys: 8.19 s, total: 56.7 s
Wall time: 41.7 s


0.9916842502509222

In [ ]:
np.mean(cross_val_score(descv, X, y, scoring="f1"))

In [ ]:
%%time
np.mean(cross_val_score(desucv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(desucv, X, y, scoring="f1", error_score="raise"))

In [ ]:
%%time
np.mean(cross_val_score(metadescv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(metadescv, X, y, scoring="f1", error_score="raise"))

In [46]:
rfc.fit(X, y)
rfc.oob_decision_function_

array([[0.90697674, 0.09302326],
       [0.96551724, 0.03448276],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ]])

In [49]:
np.argmax(rfc.oob_decision_function_, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [47]:
rfc.oob_decision_function_.shape

(569, 2)